<a href="https://colab.research.google.com/github/nijatzv/COMP5450M-Knowledge-Repres.-Reasoning/blob/main/Coursework_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KRR with Prover9 (using `pyprover9`)


## `pyprover9`
`pyprover9` is a simple interface for the Prover9 theorem prover created by **Brandon Bennett**. It is a front-end that provides functionality for running the __*Prover9*__ theorem prover direct from a __*Colab*__ notebook.

You can work in this file directly but you will not be able to save your changes. __*You should make a copy of this file so you will be able to save your own version.*__

Further information and many examples of using logic for Knowledge Representation can be found on
<a target="_blank" rel="noopener" href="https://teaching.bb-ai.net/KRR/Prover9/info.html">my Prover9 Info pages</a>.
But please work through this notebook first as explains the simplest way that you can use Prover9 to do the assignments required by the KRR module.

### The _**Prover9**_ Automated Theorem Prover

_**Prover9**_ was developed during 2006-9 based on the earlier _**Otter**_. It is an _automated theorem prover_ for _first-order logic_ and can provide general reasoning functionality that can be used in AI systems.

Although it has been around a while, Prover9 is now more powerful than ever, since we can run it on modern machines with faster CPUs and much larger memory than was avalable when it was first created.

You can find more information on the
<a target="_blank" rel="noopener" href="https://www.cs.unm.edu/~mccune/prover9/">Prover9 homepage</a>, including downloads for Windows and Linux systems.
These provide a fairly friendly GUI as well as the original Prover9 commandline program. The sofware is now very old and not maintained, so there can be some problems getting these to work, which is why I've developed this Colab interface. I have found that the GUI still works well on Windows and the Linux also works but you need old versions of Python and some other packages/libraries. I think it is hard to get it to work on recent versions of macOS.

A brief history and overview of Prover9 and ssome relevant references and links can be found on the <a target="_blank" rel="noopener" href="https://en.wikipedia.org/wiki/Prover9">Prover9 Wikipedia page</a>.





# Running Prover9 in Colab using the `pyprover9` PIP package

You can install and import `pyprover9` as shown in the code cell below.
Note that the package was designed to work in Colab and is unlikely to work in other environments. (It uses an exectuable compiled for Linux system, so it may work in other Linux-based environments.)

In [ ]:
!pip  -q install --upgrade pyprover9
import pyprover9

### How to run __Prover9__
You can now run Prover9 using the `Prover9.prove` function.

You just need to specify the assumptions and goal as strings (in the usual prover9 syntax). If you prefer you can give the assumptions as a list of strings. Prover9 expects a ful stop (`.`) after each fromula. This interface will add the stop to the end of the input strings if you omit it. However, if you put several assumptions together in a single string, you will need to ensure that there is full stop (`.`) between each of them.

In [ ]:
pyprover9.prove("(P & Q) -> R", "P -> (Q -> R)")

Here is another propositional logic example. This time there are several assumptions that should be separated by full stops.


In [ ]:
ASSUMPTIONS = """
p -> q.
q -> (r | s).
-s
"""

GOAL = "r | -p"

pyprover9.prove( ASSUMPTIONS, GOAL)

What happens if we try to prove something that cannot be proved --- i.e. is logically invalid:

In [ ]:
pyprover9.prove("P->Q. Q", "P")

## A slightly more complex example
The following example uses first order-logic representation to deduce a consequence relating to students and phones.
Notice that I have been extremely careful with the layout as it is very easy to get syntax errors (especially wrong bracketing). And you can often get bracketing or other structural issues that may not cause a syntax error but could still be wrong representations of the information you want to specify. As you see you can add comments --- text following a `%` symbol will be ignored.

I've only given the example as logical formulae but it will be a good exercise for you to look at these formulae and try to work out the intended meaning.

In [ ]:
ASSUMPTIONS = """

%%% Some general facts about students and phones:

(all x all y (Studies(x,y) -> Student(x))).

(all x ( Student(x) ->
            (exists y (Phone(y) & Owns(x,y)))
       )
).

(all x all y ( (Student(x) & Phone(y) & Owns(x,y))
               -> (Apple(y) | Android(y))
             )
).

- (exists x ( Student(x)
              & (exists y exists z (  Laptop(y) & Phone(z)
                                     & Owns(x,y) & Owns(x,z)
                                     & Apple(y) & Android(z)
                                   )
                )
            )
  ).

%%% Some particular facts about an AI student called Bing:

Studies( bing, ai ).

(exists x ( Laptop(x) & Owns( bing, x ) & Apple(x) ) ).

(exists x ( Phone(x) & Owns(bing, x) &
            HasKittenSticker( x )
          )
).
"""

GOAL = "exists x (Phone(x) & Apple(x) & HasKittenSticker(x))."

pyprover9.prove( ASSUMPTIONS, GOAL )

**Question:** Can you identify one assumption that was given but is not needed to make the deduction? Have close look at the proof.

### Time-Out and non-termination

Below is an attempt to prove something that is invalid. See what happens ...

In [ ]:
pyprover9.prove( """
                 all x (Pig(x) -> exists y Pig(y)).
                 Pig(a)
                 """,
                 "Flies(a)" )


In [ ]:
pyprover9.prove( """
                 all x (Pig(x) -> exists y Pig(y)).
                 Pig(a)
                 """,
                 "Flies(a)",
                 timeout=100)



## The Prover9 input format and `.p9` files

The `pyprover9` interface enables one to run Prover9 by just giving a list of assumption formulae and a conclusion formula. However, the Prover9 executable actually operates on an input file, which specifies configuration options regarding details of how the proof algorithm will be run. In fact there are a large range of possible options including which inference rules should be used and how formulae should be prioritised with respect to rule application choices. All the gory details are explained in the [Prover9 Manual](https://www.cs.unm.edu/~mccune/prover9/manual/2009-11A/).

Thankfully, for the relatively simple examples we are condiering in this module, we can just use simple configuration settings, and don't need to worry about the details of the Prover9 input file. However, we may sometimes want to save Prover9 input files. In particular, to do the KRR module exercises you will need to save these files and submit them to an _**autograder**_ program running on **Gradescope**. Hence, the `pyprover9` Colab interface provides functions for creating and downloading Prover9 input files (for which I use the extension `.p9`).

From a list of assumption formulas and a goal formula, you can generate a string of the corresponding `.p9` file contents as follows:

In [ ]:
p9fileContent = pyprover9.p9file( ASSUMPTIONS, GOAL )

## Take a look at the file content string:
print(p9fileContent)

## Downloading the `.p9` file

You can also download a `.p9` file constructed from your assumptions and goal as shown in the next code cell. The last argument is the file name to be used (`.p9` will be added).

In [ ]:
pyprover9.p9download(ASSUMPTIONS, GOAL, "kitten_sticker")

## Using Prover9 with an input string

Instead of giving strings containing the premiss and goal formulae. You can alternatively use the `pyprover9.input`. In this case, you just give one string representing the content of a Prover9 input file, in its usual format. This allows you to select any options, as you would be able to do with the original command line interface or the old GUI.

Assuming you've not changed directory since installing `pyprover9`, the following cell should run Prover9 on the example `diamond.p9` file that was bundled with the package.

In [ ]:
pyprover9.input(p9fileContent)

# Prover9 Exercises
<h4>Exercises</h4>
Several sets of exercises have been created
      to enable you to learn how to represent and solve
      logic problems by translating them into logical formulae
      and using the Prover9 theorem prover:
    <ul>
   <li> <a href="https://teaching.bb-ai.net/KRR/Prover9/problems/propositional/problem_index.html">
    Propositional logic problem exercises</a>
    -- you should do all of these.

   <li> <a href="https://teaching.bb-ai.net/KRR/Prover9/problems/first_order_intro/problem_index.html">
    Introductory first-order logic problem exercises</a>
   -- you should do all of these.

   <li> <a href="https://teaching.bb-ai.net/KRR/Prover9/problems/challenging/problem_index.html">
    More challenging logic problems</a>
   -- do as many as you can to develop your skills of
    logical analysis and representation.
</ul>

## Working on a Prover9 problem exercise

There are several ways you could work on one of Brandon's Prover9 logic exercises in this Colab interface.

* You can encode the list of Assumptions and Goal and use the command `pyprover9.prove( assumptions, goal )`.

* Or you can store the problem template file in a string, edit that and use the `pyprover9.input( input_string ).

* Or use the function `pyprover9.editor("probset/probname")`, which will download the problem template create a window for editing it and provie buttons for running Prover9 and for downloading your edited
`.p9` file.

Probaly the easiest is to use `pyprover9.editor`. This is illustrated in the following code cell:

In [ ]:
pyprover9.editor("propositional/diamond.p9")

In [ ]:
assumptions = [
    "DiamondStolen",
    "DiamondStolen -> ThiefInHouse",
    "ThiefInHouse -> (DoorOpen | WindowSmashed)",
    "-WindowSmashed"
]

goal = "DoorOpen"

pyprover9.prove(assumptions, goal)

You should work your way through all the first two sets of problems (`propositional` and `first_order_intro`). Try to do all the propositional logic examples before the end of week 2.

The following cells enable you to solve the problems using the `pyprover9` interface.

In [ ]:
pyprover9.editor("propositional/lost_key.p9")

In [ ]:

assumptions = [
    "-AlanLockedDoor -> (AlanForgotKey | AlanLostKey)",
    "AlanLostKey -> AlanInTrouble",
    "AlanGoodMemory -> -AlanForgotKey",
    "AlanGoodMemory"
]

goal = "AlanLockedDoor | AlanInTrouble"

pyprover9.prove(assumptions, goal)

In [ ]:
pyprover9.editor("propositional/rembrandt.p9")

In [ ]:
assumptions = [
    "PictureIsValuable | PictureIsFake",
    "PictureIsValuable -> PictureByRembrandt",
    "PictureIsFake -> PoliceInvestigation"
]

goal = "PictureByRembrandt | PoliceInvestigation"

pyprover9.prove(assumptions, goal)

In [ ]:
#pyprover9.editor("propositional/sundays.p9")

In [ ]:
assumptions = [
    "Sunday -> Holiday",
    "(Saturday & Sunday) -> Weekend",
    "Holiday -> UniversityClosed",
    "Weekend",
    "-Saturday",
    "Weekend & -Saturday -> Sunday"
]

goal = "UniversityClosed"

pyprover9.prove(assumptions, goal)

In [ ]:
pyprover9.editor("propositional/tarquin.p9")

In [ ]:
assumptions = [
    "TarquinLearnsPhilosophy -> TarquinAcquiresWisdom",
    "TarquinLearnsScience -> TarquinAdvancesKnowledge",
    "TarquinAcquiresWisdom -> -TarquinDiesMisery",
    "TarquinAdvancesKnowledge -> TarquinSatisfiesAspirations",
    "-TarquinSatisfiesAspirations -> TarquinDiesMisery",
    "-TarquinSatisfiesAspirations"
]

goal = "-TarquinLearnsPhilosophy & -TarquinLearnsScience"

pyprover9.prove(assumptions, goal)

In [ ]:
pyprover9.editor("propositional/tennis.p9")

In [ ]:
assumptions = [
    "-Playing -> Watching",
    "-Watching -> Reading",
    "-(Playing & Watching)",
    "-(Playing & Reading)",
    "-(Watching & Reading)"
]

# pyprover9.prove(assumptions, "Playing") - while running this line it said "*SEARCH FAILED* (not provable)"
# pyprover9.prove(assumptions, "Reading") - this line also gave SEARCH FAILED respond
pyprover9.prove(assumptions, "Watching")

In [ ]:
pyprover9.editor("propositional/appointment.p9")

In [ ]:
assumptions = [
    "-AlbaLate -> OpticianHappy",
    "AlbaEarly",
    "AlbaEarly -> -AlbaLate"
]

goal = "OpticianHappy"

pyprover9.prove(assumptions, goal)

In [ ]:
pyprover9.editor("propositional/dry_thunder.p9")

In [ ]:
assumptions = [
    "DryThunder -> (Lightning & -Raining)",
    "DryThunder",
    "Lightning -> (Raining | Hot)"
]

goal = "Hot"

pyprover9.prove(assumptions, goal)

In [ ]:
pyprover9.editor("propositional/dragons.p9")

In [ ]:
assumptions = [
    "DragonDestroysVillage -> (VillagersHomeless & -VillagersTakeCastle)",
    "KingMean -> VillagersTakeCastle",
    "-KingMean -> -DragonDestroysVillage",
    "DragonDestroysVillage"
]

goal = "-VillagersHomeless"

pyprover9.prove(assumptions, goal)

## First Order Logic Problems

In [ ]:
pyprover9.editor("first_order_intro/barbara.p9")

In [ ]:
pyprover9.editor("first_order_intro/celarent.p9")

In [ ]:

pyprover9.editor("first_order_intro/young_rabbits.p9")

## Getting more information about the proof run

Running the following command will print the full output from Prover9, which contains information about the initial processing of the input and also all formaule that were generated during the last run of the prover.

In [ ]:
pyprover9.full_output()

## Accessing `.p9` templates for problem exercises via `pyprover9`

The `.p9` problem template files available on Brandon's website are also included in the `pyprover9` package and can be accessed as follows:

In [ ]:
print(pyprover9.template('propositional/diamond.p9'))

### Mounting Google Drive
You can also access your Google Drive files directly in Colab by mounting your Google drive, as shown in the following code cell. (You will need to grant permission to allow this.)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

You will now be able to see your GoogleDrive files by clicking the folder icon on the left of your Colab window and using the file browser tree. You could potentially save your `.p9` files on Google drive. But it is strongly recommended that you either use the OneDrive account provided by the University of Leeds to store all your most important files.
<!--
You can aslo read a file's contents by code such as the following (assuming your have a file with path `KRR/Prover9/young_rabbits.p9` on your GoogleDrive).
-->

Once GoogleDrive is mounted, reading and writing of files can be done in the normal way. For example:

In [ ]:
with open( '/content/drive/MyDrive/testfile.txt', 'w' ) as f:
  f.write( "File written by Colab.")

with open('/content/drive/MyDrive/testfile.txt' ) as f:
  content = f.read()

print(content)

In [ ]:
###################################################################################

## Task 1

Hotspot
A logic puzzle by Brandon Bennett
Your problem is to use propositional logic to represent the following example of reasoning, and use the Prover9 theorem prover to prove that the reasoning is valid.

Use the following template file to create your Prover9 encoding of the problem: hotspot.p9

Marks: There are two marks for each sentence, making a total of 18 marks.

Outline: Losing phones and poor WiFi connections present significant challenges to today's students. Using propositional logic, we can represent troublesome situations and reason about how to fix them. The propositional logic within English sentences can be expressed in a variety of ways. To capture their meaning you need to be clear about the meaning of the logical operators and check that your formulae fit the meanings of the sentences.

In [ ]:


assumptions = """
MobileInOffice -> (MobileOnTable | MobileOnSettee).
ThinkPadOnline <-> (WiFiUp | MobileOn5G & ThinkPadInHotSpotRangeMobile).
SmartSpeakerFlashing -> -WiFiUp.
MobileInOffice | MobileInLounge | MobileInWashRoom.
ThinkPadInOffice & SmartSpeakerFlashing & IHearMusic.
(-ThinkPadOnline & IHearMusic) -> ListeningToRadio.
ThinkPadInHotSpotRangeMobile <-> -((ThinkPadInOffice & MobileInWashRoom) | (MobileInOffice & ThinkPadInWashRoom)).
-MobileOnTable & -MobileOnSettee.
"""

goal = "ListeningToRadio | MobileInLounge"

result = pyprover9.prove(assumptions, goal)



In [ ]:
pyprover9.p9download(assumptions, goal, "hotspot.p9")

In [ ]:
pyprover9.editor("/content/hotspot.p9")

In [ ]:
'''

% Saved by Prover9-Mace4 Version 0.5, December 2007.
% Last line is a lie. It is there to stop the Prover9-Mace4
% GUI giving a warning when the file is loaded.
% This file was actually created by BB Prover9 Autograder

set(ignore_option_dependencies). % GUI handles dependencies

if(Prover9). % Options for Prover9
  clear(auto).
  clear(auto_setup).
  clear(auto_limits).
  clear(auto_denials).
  clear(auto_inference).
  clear(auto_process).
  assign(eq_defs, pass).
  assign(max_seconds, 10).
  assign(max_weight, 2147483647).
  assign(sos_limit, -1).
  clear(predicate_elim).
  set(binary_resolution).
  set(paramodulation).
  set(factor).
end_if.

if(Mace4).   % Options for Mace4
  assign(max_seconds, 60).
end_if.

formulas(assumptions).

%% Hotspot
%% A logic puzzle by Brandon Bennett

%% Prover9 file: hotspot.p9

%% To solve this first-order logic proof problem, you must replace
%% each of the assumption place-holders __An__ and the goal place-holder
%% __G__, with an approapriate first-order formula that captures the
%% meaning of the given English sentence.

%% Then run Prover9.
%% If your representations are correct, it should find a Proof.
%% You can also check your representations using the Gradescope Autograder.

%% In writing the formulae, you should only use the following vocabulary:

%% Logical symbols:           &  |  -  ->   <->  =  all  exists
%% Brackets and separators:   ( )  [  ]  ,  .
%% Variables:                 Whatever you like, but must be quantified.

%% Propositions: ThinkPadInOffice, SmartSpeakerFlashing, IHearMusic, ListeningToRadio,
%%      ThinkPadOnline, WiFiUp, MobileOn5G, ThinkPadInHotSpotRangeMobile, MobileInOffice,
%%      MobileInLounge, MobileInWashRoom, ThinkPadInWashRoom, MobileOnTable,
%%      MobileOnSettee


%% Use only the specified vocabulary, otherwise the autograder will not work.



%% A1: "If the the mobile is in the office, it is either on the table or on the settee."

MobileInOffice -> (MobileOnTable | MobileOnSettee).

%% A2: "The ThinkPad is online if and only if either: (a) the WiFi is up; or (b) the
%%      mobile is connected to 5G and the mobile is in hotspot range."

ThinkPadOnline <-> (WiFiUp | MobileOn5G & ThinkPadInHotSpotRangeMobile).

%% A3: "If the smart speaker light is flashing, the WiFi is down."

SmartSpeakerFlashing -> -WiFiUp.

%% A4: "The mobile is in one of three rooms: office or lounge, or wash room."

MobileInOffice | MobileInLounge | MobileInWashRoom.

%% A5: "The ThinkPad is in the office, the smart speaker light is flashing and I hear
%%      music."

ThinkPadInOffice & SmartSpeakerFlashing & IHearMusic.

%% A6: "If ThinkPad is offline and I hear music, then I must be listening to the radio."

(-ThinkPadOnline & IHearMusic) -> ListeningToRadio.

%% A7: "The ThinkPad is always in hotspot range of my mobile except when one is in the
%%      office and the other in the wash room."

ThinkPadInHotSpotRangeMobile <-> -((ThinkPadInOffice & MobileInWashRoom) | (MobileInOffice & ThinkPadInWashRoom)).

%% A8: "The mobile is neither on the table nor on the settee."

-MobileOnTable & -MobileOnSettee.


end_of_list.

formulas(goals).

%% GOAL: "The mobile is in the lounge, unless I'm listening to the radio."

ListeningToRadio | MobileInLounge.

end_of_list.
'''

## Task 2

The Academy
A logic puzzle by Brandon Bennett
Your problem is to use first-order logic to represent the following example of reasoning, and use the Prover9 theorem prover to prove that the reasoning is valid.

Use the following template file to create your Prover9 encoding of the problem: academy.p9

Marks: There are two marks for each sentence, making a total of 22 marks.

Outline: Intellectual persuits thrive in the environment of an academic institution. Among the greatest minds there are great rivalries as well as great friendships. Use your knowledge of first-order logic to represent and reason about the qualities and relationships holding among the deep thinkers of The Academy.

*Note*: This problem is all about people. Thus, you should consider the domain of quantification to be the set of people. Hence, you do not need a Person predicate; you assume everything is a person. For example, in this context, you would represent 'Everyone loves someone' simply by (all x (exists y (Loves(x,y)))); and 'Every person loves some person' would also be represented by the same formula.

In [ ]:
pyprover9.editor("/content/academy.p9")

In [ ]:
'''

% Saved by Prover9-Mace4 Version 0.5, December 2007.
% Last line is a lie. It is there to stop the Prover9-Mace4
% GUI giving a warning when the file is loaded.
% This file was actually created by BB Prover9 Autograder

set(ignore_option_dependencies). % GUI handles dependencies

if(Prover9). % Options for Prover9
  clear(auto).
  clear(auto_setup).
  clear(auto_limits).
  clear(auto_denials).
  clear(auto_inference).
  clear(auto_process).
  assign(eq_defs, pass).
  assign(max_seconds, 10).
  assign(max_weight, 2147483647).
  assign(sos_limit, -1).
  clear(predicate_elim).
  set(binary_resolution).
  set(paramodulation).
  set(factor).
end_if.

if(Mace4).   % Options for Mace4
  assign(max_seconds, 60).
end_if.

formulas(assumptions).

%% The Academy
%% A logic puzzle by Brandon Bennett

%% Prover9 file: academy.p9

%% To solve this first-order logic proof problem, you must replace
%% each of the assumption place-holders __An__ and the goal place-holder
%% __G__, with an approapriate first-order formula that captures the
%% meaning of the given English sentence.

%% Then run Prover9.
%% If your representations are correct, it should find a Proof.
%% You can also check your representations using the Gradescope Autograder.

%% In writing the formulae, you should only use the following vocabulary:

%% Logical symbols:           &  |  -  ->   <->  =  all  exists
%% Brackets and separators:   ( )  [  ]  ,  .
%% Variables:                 Whatever you like, but must be quantified.

%% Names: thales, theano
%% Properties: Happy, KRR_Expert, Mathematician, Playwright, Painter, DreamsOfSuccess
%% Relations: Taller, Shorter, Idolises, Imitates


%% Use only the specified vocabulary, otherwise the autograder will not work.



%% A1: "At least two painters are shorter than Theano."

(exists x exists y (Painter(x) & Painter(y) & x != y & Shorter(x, theano) & Shorter(y, theano))).

%% A2: "Every playwright who dreams of success is not happy."

all x (Playwright(x) & DreamsOfSuccess(x) -> -Happy(x)).

%% A3: "Everyone who imitates a KRR expert is either a playwright or a mathematician."

all x all y (Imitates(x, y) & KRR_Expert(y) -> (Playwright(x) | Mathematician(x))).

%% A4: "Every painter idolises exactly one KRR expert (they do not necessarily idolise the
%%      same one)."

all x (Painter(x) -> (exists y (KRR_Expert(y) & Idolises(x, y) & all z (KRR_Expert(z) & Idolises(x, z) -> z = y)))).

%% A5: "Idolisation implies imitation."

all x all y (Idolises(x, y) -> Imitates(x, y)).

%% A6: "Among KRR experts, only Thales is idolised by someone shorter than Theano."

all x all y (KRR_Expert(x) & Idolises(y, x) & Shorter(y, theano) -> x = thales).

%% A7: "Thales is the only KRR expert who is taller than Theano."

all x (KRR_Expert(x) & Taller(x, theano) -> x = thales) & KRR_Expert(thales) & Taller(thales, theano).

%% A8: (*Amended version*)
%%     "If someone is taller than any person, then they are taller than everyone who
%%      is shorter than that person."

all x all y all z (Taller(x, y) & Shorter(z, y) -> Taller(x, z)).

%% A9: "No mathematician is a painter."

all x (Mathematician(x) -> -Painter(x)).

%% A10: "Anyone who idolises someone taller than themself dreams of success."

all x all y (Idolises(x, y) & Taller(y, x) -> DreamsOfSuccess(x)).


end_of_list.

formulas(goals).

%% GOAL: "Some playwright is not happy."

exists x (Playwright(x) & -Happy(x)).

end_of_list.
'''